In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import requests # 웹페이지를 불러오기 위한 패키지
import lxml.html
from selenium import webdriver
from bs4 import BeautifulSoup # HTML 문서에서 원하는 데이터를 파싱하기 위한 모듈
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

소설 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=01&menuCode=002

시/에세이 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=03&menuCode=002

인문 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=05&menuCode=002

가정/육아 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=07&menuCode=002

요리 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=08&menuCode=002

건강 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=09&menuCode=002

취미/실용/스포츠 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=11&menuCode=002

경제/경영 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=13&menuCode=002

자기계발 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=15&menuCode=002

정치/사회 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=17&menuCode=002

역사/문화 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=19&menuCode=002

종교 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=21&menuCode=002

예술/대중문화 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=23&menuCode=002

중/고등 참고서 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=25&menuCode=002

기술/공학 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=26&menuCode=002

외국어 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=27&menuCode=002

과학 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=29&menuCode=002

취업/수험서 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=31&menuCode=002

여행 : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=32&menuCode=002

컴퓨터/IT : http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=33&menuCode=002

In [ ]:
# 교보문고의 소설 카테고리의 베스트셀러 웹페이지를 가져옵니다.
url= "http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=33&menuCode=002"
html = requests.get(url).text

soup = BeautifulSoup(html, "html5lib") # html5lib은 html 데이터와 HTML문서를 파싱하는데 사용할 모듈

#chrome 옵션
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get("http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?perPage=20&mallGb=KOR&linkClass=33&menuCode=002")
keyword_lists = []

# keyword_counts = []

#pageNum = 페이지에 관한 태그(html)가 들어있는 배열
pageNum = soup.select("div.list_paging ul li a")

# n  = 페이지 번호 - 1
for n in range(0, 7):
  number = n+1
  # print(str(number)+"페이지")
  pageNum[n]
  # 1페이지가 아닌 페이지를 봐야할 경우, 클릭을 통해 해당 페이지로 넘어감
  if pageNum[n].text != "1":
    page_path = "/html/body/div/div[1]/div[2]/div[2]/div/div[2]/div/div[4]/div[1]/div/ul/li["+str(number)+"]/a"
    page = driver.find_element_by_xpath(page_path)
    page.click()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
  
  #titles = 타이틀 태그, scoreReviews = 평점 태그, numReviews = 리뷰 개수 태그
  titles = soup.select("ul.prd_list_type1 li.id_detailli .title")
  scoreReviews = soup.select("ul.prd_list_type1 li.id_detailli .score")
  numReviews = soup.select("ul.prd_list_type1 li.id_detailli .review")
  # title_list = []
  # score_list = []
  # review_list = []
  all = []
  # i = 한 페이지 내의 책 순서(?) ex. 1위-20위, 21위-40위....
  for i in range(0, 20):
    # nnnn = i+1
    total = []
    title = titles[i].text.strip()
    # title_list.append(title)

    scores = scoreReviews[i].text.strip()
    scores = scores.strip().replace(" ", "")
    scores = scores.replace("\t","")
    scores = scores.replace("\n","")
    scores = scores.replace("/10", "")
    # score_list.append(scores)

    review = numReviews[i].text.replace("리뷰", "").strip()
    # review_list.append(review)

    total.append(title)
    total.append(scores)
    total.append(review)

    all.append(total)
    # print("   "+str(nnnn)+"번째 책")
    # print("       Title : " + title)
    # print("       Score : " + scores)
    # print("       Review : " + review)

# b = 한 페이지 내에서 키워드를 읽기 위한 책 순서(?)
# 책마다 동일한 태그가 2번씩 있기 때문에 홀수 태그만 읽으면 된다
  for b in range(1,40,2):
    try:
      x_path = "/html/body/div/div[1]/div[2]/div[2]/div/div[2]/div/div[3]/form/ul/li[" + str(b) + "]/div/div[1]/div[2]/div[1]/a"
      list = driver.find_element_by_xpath(x_path)
      list.click()

      alert = driver.switch_to.alert
      alert.accept()
      continue

    except: # 19세 책이 아닐 경우 
      html = driver.page_source
      soup = BeautifulSoup(html, 'html.parser')
      keyword_list = []

      for anchor in soup.select("div.book_keyword a"):
        keyword_list.append(anchor.get_text())
        # keyword_counts.append(anchor.get_text())

      all[b//2].append(keyword_list)
      driver.back()
  
  # print(number)
  # print(all)

  # keyword_counts.value_counts().head(20)
  # print(keyword_counts)

  # csv 파일에 넣어주는 과정
  with open('/content/drive/Shareddrives/데이터마이닝/computer.csv','a', newline='') as file :

    write = csv.writer(file)
    write.writerows(all)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


1
[['비전공자를 위한 이해할 수 있는 IT 지식', '8.5', '2555', ['파이썬', '프레임워크', '네트워크', '클라', '자바', '프로그래밍 언어']], ['혼자 공부하는 파이썬', '9.6', '294', ['프로그래밍', '딕셔너리', '반 복문', '메소드', '자습서', '리스트']], ['2022 시나공 총정리 컴퓨터활용능력 1급 필기', '8.3', '12', ['실전 모의고사', '엑셀 기능', '실습', '핵심 요약', '원서 접수 방법', '답변 데이터']], ['Do it! 점프 투 파이썬', '9.3', '156', ['정규 표현식', 'Python', '딕셔너리', '튜플', '리스트', '서드']], ['2022 빅데이터 분석 기사 실기 필답형+작업형', '9.2', '3', ['전처리', '내기', '인터프리터', '실습', '내장', '탐색']], ['2022 시나공 컴퓨터활용능력 1급 실기 엑셀, 엑세스 2016', '10', '25', ['실전 모의고사', '기출문제', '삼각 함수', '기본 모의고사', '쿼리', '최신']], ['혼자 공부하는 머신러닝+딥러닝', '9.7', '52', ['패션', '인공지능', '로지스틱 회귀', '인공 신경망', '딥러닝', '핵심 포인트']], ['회사에서 바로 통하는 실무 엑셀+파워포인트+워드&한글', '9.5', '116', ['핵심 기능', '우선순위', '글꼴', '조건부 서식 지정', '스타일 적용', '서식 지정']], ['전산회계 1급 이론+실기+기출문제(2021)(해커스)(개정판)', '8.7', '149', ['부가가치세 총칙', '원가회계', '단기 합격', '손익계산서', '전표', '빈출']], ['클린 아키텍처(만들면서 배우는)(위키북스 IT Leaders 시리즈 34)', '0', '0', ['매핑', '도메인', '어댑터', '단위 테스트', '빌드', '애플리케이션']], ['2022 시나공 컴퓨터활용능력 2급 실기 엑셀2016', '10'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


2
[['2022 시나공 총정리 컴퓨터활용능력 2급 필기', '5.8', '3', ['실전 모의고사', '엑셀 기능', '핵심 요약', '실습', '원서 접수 방법', '답변 데이터']], ['2022 시나공 컴퓨터활용능력 1급 필기', '10', '18', []], ['파이썬 알고리즘 인터뷰', '9.8', '80', ['프로그래밍', '윈도우', '알고리즘', '딕셔너리', '연결 리스트', '취업']], ['모던 자바스크립트 Deep Dive(위키북스 프로그래밍 & 프랙티스 시리즈 26)', '10', '33', ['메 서드', '프로토타입', '정규 표현식', '리터럴', '파싱', '스코프']], ['2022 이기적 컴퓨터활용능력 2급 필기 기본서(전2권)', '10', '5', ['데이터 관리', '매크로', '스프레드시트 일반', '기출 변형 문제', '최신 기출문제', '데이터 입력']], ['2022 시나공 워드프로세서 실기 한글 2020', '10', '4', ['실전 모의고사', '최신 기출문제', '자동 채점 프로그램', '하이퍼링크', '다단 설정', '한자 변환']], ['실전 카프카 개발부터 운영까지', '0', '0', ['빅데이터', '파이썬', '레지스트리', '엔터', '데이터분석', '이용한']], ['리팩터링(2판)', '10', '38', ['생성자', '알고리즘', '매개변수', '상속', '로직', '리터럴']], ['머신러닝 디자인 패턴', '10', '5', ['워크플로', '솔루션', '스테이', '데이터 표현', '클래스', '모달']], ['한 권으로 읽는 컴퓨터 구조와 프로그래밍', '9', '26', ['인코딩', '프로그래밍언어', '마크업', '전자회로', '소프트웨어개발', '리팩토링']], ['2022 이기적 컴퓨터활용능력 1급 필기 기본서 세트(전3권)', '10', '13', ['프로그래밍', '기출문제', '쿼리', '데이터 관리', '매크로', '스프레드시트 일반']], ['클린 아키텍처: 

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/데이터마이닝/computer.csv', delimiter=',', names=["title", "score", "review_num", "keyword"])
df["category"] = "컴퓨터/IT"

df_cleaned = df.dropna(axis = 0) # 모든 열에 NaN이 있다면 삭제
df_cleaned.reset_index(inplace=True, drop=True)
df_cleaned = df_cleaned[["title", "score", "review_num", "keyword", "category"]]

df_cleaned

df_cleaned.to_csv("/content/drive/Shareddrives/데이터마이닝/computer_cleaned.csv", mode='w')